***GENERATED CODE FOR joinwithdataprep PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv("hdfs://34.83.203.43:54310"+eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***OPERATION FUNCTIONS***

In [ ]:


def QueryEditor(spark, joindataobj):
    for source in joindataobj["sourcesForQuery"]:
        sourcename = source['sourcename']
        source['sourcename'] = spark.read.options(header='true', inferschema='true', delimiter=',')\
            .csv('dbfs:' + source['sourcepath'])
        source['sourcename'].createOrReplaceTempView(sourcename)
    joinedDf = spark.sql(joindataobj['querytext'])
    df = removeIfDuplicates(joinedDf)
    print(display(df.limit(2).toPandas()))
    return df


def removeIfDuplicates(joinedDf):
    cols_new = []
    seen = set()
    for c in joinedDf.columns:
        cols_new.append('{}_dup'.format(c) if c in seen else c)
        seen.add(c)
    df = joinedDf.toDF(*cols_new).select(*
                                         [c for c in cols_new if not c.endswith('_dup')])
    return df


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
import json
from dask.dataframe import from_pandas


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.functions import round
from pyspark.ml.feature import Binarizer
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def BinarizerTransform(df, params, transformationData={}):
    dfReturn = df
    transform_params = params
    feature = transform_params['feature']
    outcol = feature + "_binarizer"
    dfReturn = dfReturn.withColumn("feature_cast", dfReturn[feature].cast("double")).drop(feature)\
        .withColumnRenamed("feature_cast", feature)

    dfReturn = dfReturn.fillna({feature: 0.0})
    binarizer = Binarizer(threshold=float(
        transformationData['threshold']), inputCol=feature, outputCol=outcol)
    binarizedDataFrame = binarizer.transform(dfReturn)

    # binarizedDataFrame=binarizedDataFrame.drop(feature).withColumnRenamed(outcol,feature)

    dfReturn = binarizedDataFrame
    dfReturn = dfReturn.withColumn(feature, round(dfReturn[feature], 2))

    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = BinarizerTransform(transformationDF, {'transformationsData': [{'feature_label': 'price_sum', 'threshold': 243681.36550448235, 'transformation_label': 'Binarizer'}], 'feature': 'price_sum', 'type': 'real', 'selected': 'True', 'replaceby': 'mean', 'stats': {
                                              'count': '12933', 'mean': '243681.37', 'stddev': '1388278.53', 'min': '70.88', 'max': '1.5953485738516184E7', 'missing': '0'}, 'transformation': [{'transformation': 'Binarizer', 'selectedAsDefault': 1}], 'updatedLabel': 'price_sum'}, {'feature_label': 'price_sum', 'threshold': 243681.36550448235, 'transformation_label': 'Binarizer'})
        transformationDF = transformationDF.drop('price_sum')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'start_date', 'transformation_label': 'Extract Date'}], 'feature': 'start_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'start_date'}, {'feature_label': 'start_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('start_date')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'end_date', 'transformation_label': 'Extract Date'}], 'feature': 'end_date', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'end_date'}, {'feature_label': 'end_date', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('end_date')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run joinwithdataprepHooks.ipynb
try:
	#sourcePreExecutionHook()
		train = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/couponRedemption/train.csv', 'filename': 'train.csv', 'token': '', 'delimiter': ','}")


		coupon = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/couponRedemption/coupon.csv', 'filename': 'coupon.csv', 'token': '', 'delimiter': ','}")


		transactions = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/couponRedemption/transactions.csv', 'filename': 'transactions.csv', 'token': '', 'delimiter': ','}")


		custdemo = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/couponRedemption/cust_demo.csv', 'filename': 'cust_demo.csv', 'token': '', 'delimiter': ','}")



	campaign = HDFSConnector.fetch(spark, "{'is_header': 'Use Header Line', 'file_type': 'Delimeted', 'domain': '', 'url': '/FileStore/platform/Data/TrainData/couponRedemption/campaign.csv', 'filename': 'campaign.csv', 'token': '', 'delimiter': ','}")
	#sourcePostExecutionHook(campaign)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#operationPreExecutionHook()

	join = QueryEditor(spark, {"querytext": "SELECT *\r\nFROM train\r\nJOIN coupon\r\nON train.coupon_id = coupon.coupon_id\r\nJOIN campaign\r\nON train.campaign_id = campaign.campaign_id\r\nJOIN cust_demo\r\nON train.customer_id = cust_demo.customer_id\r\nJOIN transactions\r\nON cust_demo.customer_id = transactions.customer_id", "sourcesForQuery": [{"sourcename": "campaign", "sourcepath": "/FileStore/platform/Data/TrainData/couponRedemption/campaign.csv", "sourceid": "5f9fbacb5d8040225bd31ae1"}, {"sourcename": "train", "sourcepath": "/FileStore/platform/Data/TrainData/couponRedemption/train.csv", "sourceid": "5f9fbaa75d8040225bd31adb"}, {"sourcename": "cust_demo", "sourcepath": "/FileStore/platform/Data/TrainData/couponRedemption/cust_demo.csv", "sourceid": "5f9fbaca5d8040225bd31ae0"}, {"sourcename": "coupon", "sourcepath": "/FileStore/platform/Data/TrainData/couponRedemption/coupon.csv", "sourceid": "5f9fbac05d8040225bd31ade"}, {"sourcename": "transactions", "sourcepath": "/FileStore/platform/Data/TrainData/couponRedemption/transactions.csv", "sourceid": "5f9fbac45d8040225bd31adf"}], "fieldsForQuery": [{"sourcename": "campaign", "fieldname": "campaign_id"}, {"sourcename": "train", "fieldname": "id"}, {"sourcename": "train", "fieldname": "campaign_id"}, {"sourcename": "train", "fieldname": "coupon_id"}, {"sourcename": "train", "fieldname": "customer_id"}, {"sourcename": "cust_demo", "fieldname": "customer_id"}, {"sourcename": "coupon", "fieldname": "coupon_id"}, {"sourcename": "transactions", "fieldname": "customer_id"}]})
	#operationPostExecutionHook(join)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(join,spark,json.dumps( {"url": "/FileStore/platform/sampleData/5f9fbae25d8040225bd31ae2JoinStage/part-00000-e54cd3e8-2baa-44ae-8936-36a97c2f940d-c000.csv", "source_attributes": null, "data_source": "hdfs", "BasicStats": {"missingValues": 0, "numberOfColumns": 40, "numberOfRows": 43661, "duplicateRowCount": 0, "stats": [{"column": "id", "alias": "id", "type": "numeric", "max": 128592, "min": 1, "mean": 64303.83028331921, "missing": 0, "stddev": 37166.364202520694}, {"column": "campaign_id", "alias": "campaign_id", "type": "numeric", "max": 30, "min": 1, "mean": 13.750830260415475, "missing": 0, "stddev": 7.839749022556213}, {"column": "coupon_id", "alias": "coupon_id", "type": "numeric", "max": 1115, "min": 1, "mean": 568.462792881519, "missing": 0, "stddev": 329.64723801385884}, {"column": "customer_id", "alias": "customer_id", "type": "numeric", "max": 1581, "min": 1, "mean": 780.4545246329676, "missing": 0, "stddev": 461.87184228258633}, {"column": "redemption_status", "alias": "redemption_status", "type": "numeric", "max": 1, "min": 0, "mean": 0.013009321820388905, "missing": 0, "stddev": 0.11331537167869658}, {"column": "brand", "alias": "brand", "type": "numeric", "max": 5395, "min": 1, "mean": 1062.1794736721558, "missing": 0, "stddev": 1338.6538048350535}, {"column": "brand_type", "alias": "brand_type", "type": "numeric", "max": 1, "min": 0, "mean": 0.20901949107899498, "missing": 0, "stddev": 0.406612998067298}, {"column": "category", "alias": "category", "type": "numeric", "max": 16, "min": 1, "mean": 7.439591397356909, "missing": 0, "stddev": 2.5702405650661766}, {"column": "cd_sum", "alias": "cd_sum", "type": "real", "max": 0, "min": -113627.33999999987, "mean": -1467.6008728613629, "missing": 0, "stddev": 7435.927963011159}, {"column": "coupon_discount", "alias": "coupon_discount", "type": "real", "max": 0, "min": -71.24, "mean": -2.28963647655418, "missing": 0, "stddev": 5.8191938294200956}, {"column": "coupon_used", "alias": "coupon_used", "type": "real", "max": 2661, "min": 0, "mean": 43.83152012093172, "missing": 0, "stddev": 213.29177434825488}, {"column": "item_counts", "alias": "item_counts", "type": "numeric", "max": 11813, "min": 1, "mean": 87.21412702411763, "missing": 0, "stddev": 644.3452150371244}, {"column": "no_of_customers", "alias": "no_of_customers", "type": "real", "max": 118.57142857142857, "min": 1, "mean": 17.489052972862932, "missing": 0, "stddev": 13.563678094123928}, {"column": "od_sum", "alias": "od_sum", "type": "real", "max": 0, "min": -3259657.797895498, "mean": -36112.837875168036, "missing": 0, "stddev": 221782.74764481932}, {"column": "other_discount", "alias": "other_discount", "type": "real", "max": 0, "min": -481.4993333333333, "mean": -20.733530332989062, "missing": 0, "stddev": 20.415496496468364}, {"column": "price_sum", "alias": "price_sum", "type": "real", "max": 15953485.738516184, "min": 70.88, "mean": 224296.8891037507, "missing": 0, "stddev": 1301047.8462737848}, {"column": "qu_sum", "alias": "qu_sum", "type": "real", "max": 207579, "min": 1, "mean": 2697.5057373857676, "missing": 0, "stddev": 16017.15479607203}, {"column": "quantity", "alias": "quantity", "type": "real", "max": 4.096091911309303, "min": 1, "mean": 1.1739291066705713, "missing": 0, "stddev": 0.2648396656395889}, {"column": "selling_price", "alias": "selling_price", "type": "real", "max": 1235.863616557734, "min": 13.416928746729855, "mean": 161.4864360756459, "missing": 0, "stddev": 141.35023639995546}, {"column": "t_counts", "alias": "t_counts", "type": "real", "max": 158788, "min": 1, "mean": 2089.96216302879, "missing": 0, "stddev": 12140.662558420456}, {"column": "total_discount_mean", "alias": "total_discount_mean", "type": "real", "max": 0, "min": -508.808, "mean": -23.02316680954324, "missing": 0, "stddev": 21.947583526238724}, {"column": "total_discount_sum", "alias": "total_discount_sum", "type": "real", "max": 0, "min": -3344221.267895495, "mean": -37580.4387480294, "missing": 0, "stddev": 227639.07543610383}, {"column": "campaign_type", "alias": "campaign_type", "type": "numeric", "max": 1, "min": 0, "mean": 0.2806852797691303, "missing": 0, "stddev": 0.44933915685628567}, {"column": "start_date", "alias": "start_date", "type": "date", "max": "2013519", "min": "2012812", "mean": "", "missing": 0, "stddev": ""}, {"column": "end_date", "alias": "end_date", "type": "date", "max": "201375", "min": "2012921", "mean": "", "missing": 0, "stddev": ""}, {"column": "campaign_duration", "alias": "campaign_duration", "type": "numeric", "max": 63, "min": 32, "mean": 44.68276035821442, "missing": 0, "stddev": 6.31080574968826}, {"column": "age_range", "alias": "age_range", "type": "numeric", "max": 5, "min": 0, "mean": 2.4339341746638876, "missing": 0, "stddev": 1.2435696547377482}, {"column": "marital_status", "alias": "marital_status", "type": "numeric", "max": 1, "min": 0, "mean": 0.3947687867891253, "missing": 0, "stddev": 0.488806571353548}, {"column": "rented", "alias": "rented", "type": "numeric", "max": 1, "min": 0, "mean": 0.05737385767618699, "missing": 0, "stddev": 0.23255824397769043}, {"column": "family_size", "alias": "family_size", "type": "numeric", "max": 5, "min": 1, "mean": 2.233572295641419, "missing": 0, "stddev": 1.209293961651159}, {"column": "no_of_children", "alias": "no_of_children", "type": "numeric", "max": 3, "min": 0, "mean": 0.6169350221021048, "missing": 0, "stddev": 0.9717286079504597}, {"column": "income_bracket", "alias": "income_bracket", "type": "numeric", "max": 12, "min": 1, "mean": 4.802478184191842, "missing": 0, "stddev": 2.344018869069544}, {"column": "day", "alias": "day", "type": "numeric", "max": 31, "min": 1, "mean": 15.210851789926936, "missing": 0, "stddev": 8.591874868282252}, {"column": "dow", "alias": "dow", "type": "numeric", "max": 6, "min": 0, "mean": 3.2424818487895375, "missing": 0, "stddev": 1.7036233963963654}, {"column": "no_of_items", "alias": "no_of_items", "type": "numeric", "max": 2088, "min": 97, "mean": 668.3454570440439, "missing": 0, "stddev": 333.22589517694666}, {"column": "month", "alias": "month", "type": "numeric", "max": 12, "min": 1, "mean": 4.892764709924188, "missing": 0, "stddev": 2.1562526895725163}, {"column": "cdd_sum", "alias": "cdd_sum", "type": "real", "max": 0, "min": -11937.8, "mean": -892.7248587984701, "missing": 0, "stddev": 1529.5084817347006}, {"column": "customer_id_count", "alias": "customer_id_count", "type": "numeric", "max": 4134, "min": 144, "mean": 1197.0961269783102, "missing": 0, "stddev": 669.3985343165846}, {"column": "odd_sum", "alias": "odd_sum", "type": "real", "max": -873.175, "min": -62321.37241, "mean": -16312.650187993131, "missing": 0, "stddev": 9877.92399953933}, {"column": "qa_sum", "alias": "qa_sum", "type": "numeric", "max": 2617544, "min": 179, "mean": 183827.78972080347, "missing": 0, "stddev": 276185.91965389287}]}, "startListenerOnly": 1, "FilePath": "/FileStore/platform/sampleData/5f9fbae45d8040225bd31ae3_DataPreparation/0part.csv", "HasBasicStats": 1, "functionsApplied": [], "fileheader": [{"field": "id", "alias": "id", "position": 1, "type": "numeric"}, {"field": "campaign_id", "alias": "campaign_id", "position": 2, "type": "numeric"}, {"field": "coupon_id", "alias": "coupon_id", "position": 3, "type": "numeric"}, {"field": "customer_id", "alias": "customer_id", "position": 4, "type": "numeric"}, {"field": "redemption_status", "alias": "redemption_status", "position": 5, "type": "numeric"}, {"field": "brand", "alias": "brand", "position": 6, "type": "numeric"}, {"field": "brand_type", "alias": "brand_type", "position": 7, "type": "numeric"}, {"field": "category", "alias": "category", "position": 8, "type": "numeric"}, {"field": "cd_sum", "alias": "cd_sum", "position": 9, "type": "real"}, {"field": "coupon_discount", "alias": "coupon_discount", "position": 10, "type": "real"}, {"field": "coupon_used", "alias": "coupon_used", "position": 11, "type": "real"}, {"field": "item_counts", "alias": "item_counts", "position": 12, "type": "numeric"}, {"field": "no_of_customers", "alias": "no_of_customers", "position": 13, "type": "real"}, {"field": "od_sum", "alias": "od_sum", "position": 14, "type": "real"}, {"field": "other_discount", "alias": "other_discount", "position": 15, "type": "real"}, {"field": "price_sum", "alias": "price_sum", "position": 16, "type": "real"}, {"field": "qu_sum", "alias": "qu_sum", "position": 17, "type": "real"}, {"field": "quantity", "alias": "quantity", "position": 18, "type": "real"}, {"field": "selling_price", "alias": "selling_price", "position": 19, "type": "real"}, {"field": "t_counts", "alias": "t_counts", "position": 20, "type": "real"}, {"field": "total_discount_mean", "alias": "total_discount_mean", "position": 21, "type": "real"}, {"field": "total_discount_sum", "alias": "total_discount_sum", "position": 22, "type": "real"}, {"field": "campaign_type", "alias": "campaign_type", "position": 23, "type": "numeric"}, {"field": "start_date", "alias": "start_date", "position": 24, "type": "date"}, {"field": "end_date", "alias": "end_date", "position": 25, "type": "date"}, {"field": "campaign_duration", "alias": "campaign_duration", "position": 26, "type": "numeric"}, {"field": "age_range", "alias": "age_range", "position": 27, "type": "numeric"}, {"field": "marital_status", "alias": "marital_status", "position": 28, "type": "numeric"}, {"field": "rented", "alias": "rented", "position": 29, "type": "numeric"}, {"field": "family_size", "alias": "family_size", "position": 30, "type": "numeric"}, {"field": "no_of_children", "alias": "no_of_children", "position": 31, "type": "numeric"}, {"field": "income_bracket", "alias": "income_bracket", "position": 32, "type": "numeric"}, {"field": "day", "alias": "day", "position": 33, "type": "numeric"}, {"field": "dow", "alias": "dow", "position": 34, "type": "numeric"}, {"field": "no_of_items", "alias": "no_of_items", "position": 35, "type": "numeric"}, {"field": "month", "alias": "month", "position": 36, "type": "numeric"}, {"field": "cdd_sum", "alias": "cdd_sum", "position": 37, "type": "real"}, {"field": "customer_id_count", "alias": "customer_id_count", "position": 38, "type": "numeric"}, {"field": "odd_sum", "alias": "odd_sum", "position": 39, "type": "real"}, {"field": "qa_sum", "alias": "qa_sum", "position": 40, "type": "numeric"}], "requestRatio": 87, "totalRows": 43661, "functionChanges": []}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "63874.26", "stddev": "37199.28", "min": "15", "max": "128587", "missing": "0"}, "updatedLabel": "id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "13.81", "stddev": "7.89", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "campaign_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "569.99", "stddev": "329.4", "min": "1", "max": "1115", "missing": "0"}, "updatedLabel": "coupon_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "781.05", "stddev": "464.72", "min": "1", "max": "1581", "missing": "0"}, "updatedLabel": "customer_id"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "redemption_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.01", "stddev": "0.12", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "redemption_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "1080.63", "stddev": "1357.03", "min": "1", "max": "5395", "missing": "0"}, "updatedLabel": "brand"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "brand_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.21", "stddev": "0.41", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "brand_type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "category", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "7.45", "stddev": "2.57", "min": "1", "max": "16", "missing": "0"}, "updatedLabel": "category"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-1579.41", "stddev": "8014.84", "min": "-113627.33999999987", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-2.34", "stddev": "5.97", "min": "-71.24", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_discount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "coupon_used", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "46.97", "stddev": "228.94", "min": "0.0", "max": "2661.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "coupon_used"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "item_counts", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "96.05", "stddev": "696.71", "min": "1", "max": "11813", "missing": "0"}, "updatedLabel": "item_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_customers", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "17.41", "stddev": "13.64", "min": "1.0", "max": "118.57142857142857", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "no_of_customers"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "od_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-39293.21", "stddev": "235375.77", "min": "-3259657.797895498", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "od_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "other_discount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-20.52", "stddev": "19.83", "min": "-481.4993333333333", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "other_discount"}, {"transformationsData": [{"feature_label": "price_sum", "threshold": 243681.36550448235, "transformation_label": "Binarizer"}], "feature": "price_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "243681.37", "stddev": "1388278.53", "min": "70.88", "max": "1.5953485738516184E7", "missing": "0"}, "transformation": [{"transformation": "Binarizer", "selectedAsDefault": 1}], "updatedLabel": "price_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qu_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "2916.91", "stddev": "17092.59", "min": "1.0", "max": "207579.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "qu_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "quantity", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "1.17", "stddev": "0.26", "min": "1.0", "max": "4.096091911309303", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "quantity"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "selling_price", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "161.79", "stddev": "143.08", "min": "13.416928746729855", "max": "1235.863616557734", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "selling_price"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "t_counts", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "2258.01", "stddev": "12964.69", "min": "1.0", "max": "158788.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "t_counts"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_mean", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-22.86", "stddev": "21.4", "min": "-508.808", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "total_discount_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-40872.62", "stddev": "241643.56", "min": "-3344221.267895495", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "total_discount_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_type", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.28", "stddev": "0.45", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "campaign_type"}, {"transformationsData": [{"feature_label": "start_date", "transformation_label": "Extract Date"}], "feature": "start_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "start_date"}, {"transformationsData": [{"feature_label": "end_date", "transformation_label": "Extract Date"}], "feature": "end_date", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "end_date"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "campaign_duration", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "44.73", "stddev": "6.28", "min": "32", "max": "63", "missing": "0"}, "updatedLabel": "campaign_duration"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "age_range", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "2.43", "stddev": "1.25", "min": "0", "max": "5", "missing": "0"}, "updatedLabel": "age_range"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "marital_status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.4", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "marital_status"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rented", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.05", "stddev": "0.23", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "rented"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "family_size", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "2.23", "stddev": "1.21", "min": "1", "max": "5", "missing": "0"}, "updatedLabel": "family_size"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_children", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "0.62", "stddev": "0.97", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "no_of_children"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "income_bracket", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "4.8", "stddev": "2.35", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "income_bracket"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "day", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "15.21", "stddev": "8.57", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "day"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dow", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "3.24", "stddev": "1.71", "min": "0", "max": "6", "missing": "0"}, "updatedLabel": "dow"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "no_of_items", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "667.18", "stddev": "329.31", "min": "97", "max": "2088", "missing": "0"}, "updatedLabel": "no_of_items"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "4.92", "stddev": "2.17", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "month"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "cdd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-876.53", "stddev": "1500.21", "min": "-11937.8", "max": "0.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "cdd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "customer_id_count", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "1194.23", "stddev": "668.42", "min": "144", "max": "4134", "missing": "0"}, "updatedLabel": "customer_id_count"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "odd_sum", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "12933", "mean": "-16228.15", "stddev": "9806.86", "min": "-62321.37241", "max": "-873.175", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "odd_sum"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "qa_sum", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "12933", "mean": "183675.62", "stddev": "279227.69", "min": "179", "max": "2617544", "missing": "0"}, "updatedLabel": "qa_sum"}, {"feature": "price_sum_binarizer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "12933", "mean": "0.09", "stddev": "0.29", "min": "0.0", "max": "1.0", "missing": "0"}, "updatedLabel": "price_sum_binarizer_trans..."}, {"feature": "start_date_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "15.98", "stddev": "4.41", "min": "2", "max": "28", "missing": "0"}, "updatedLabel": "start_date_dayofmonth"}, {"feature": "start_date_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "4.83", "stddev": "3.02", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "start_date_month"}, {"feature": "start_date_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "2012.77", "stddev": "0.42", "min": "2012", "max": "2013", "missing": "0"}, "updatedLabel": "start_date_year"}, {"feature": "end_date_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "9.19", "stddev": "7.41", "min": "1", "max": "30", "missing": "0"}, "updatedLabel": "end_date_dayofmonth"}, {"feature": "end_date_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "5.46", "stddev": "2.56", "min": "1", "max": "11", "missing": "0"}, "updatedLabel": "end_date_month"}, {"feature": "end_date_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "12933", "mean": "2012.86", "stddev": "0.34", "min": "2012", "max": "2013", "missing": "0"}, "updatedLabel": "end_date_year"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run joinwithdataprepHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["id", "campaign_id", "coupon_id", "customer_id", "brand", "brand_type", "category", "cd_sum", "coupon_discount", "coupon_used", "item_counts", "no_of_customers", "od_sum", "other_discount", "price_sum_binarizer", "qu_sum", "quantity", "selling_price", "t_counts", "total_discount_mean", "total_discount_sum", "campaign_type", "campaign_duration", "age_range", "marital_status", "rented", "family_size", "no_of_children", "income_bracket", "day", "dow", "no_of_items", "month", "cdd_sum", "customer_id_count", "odd_sum", "qa_sum"], "redemption_status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

